In [18]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import make_scorer, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier

from sklearn.cluster import KMeans

In [2]:
df = pd.read_csv('./data/Frogs_MFCCs.csv')
df.head()

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1


In [3]:
df['Family'] = df['Family'].apply(lambda x: 1 if (x == 'Dendrobatidae') else 0)

In [4]:
drop_cols = ['Family', 'Genus', 'Species', 'RecordID']
X = df.drop(drop_cols, axis=1)
y = df['Family']

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=31)

In [5]:
y.value_counts()

0    6653
1     542
Name: Family, dtype: int64

In [6]:
# Задание 7.5

# Для начала обучите на наших данных случайный лес с десятью решающими деревьями. Воспользуйтесь параметрами по умолчанию. В
# качестве значения random state возьмите число 42.
# Оцените значение F1 -меры и введите его в качестве ответа, предварительно округлив до двух знаков после точки-разделителя.

forest = RandomForestClassifier(n_estimators=10, random_state=42)
forest.fit(X_train, y_train)

y_forest = forest.predict(X_test)
print(f1_score(y_test, y_forest).round(2))


0.97


In [9]:
# Задание 7.6

# Теперь попробуем улучшить качество нашего предсказания за счёт использования стекинга.
# В качестве базовых моделей выберите следующие:
# • случайный лес с десятью деревьями, random state = 31;
# • KNN, количество соседей = 11:
# • наивный байесовский классификатор с параметрами по умолчанию, в качестве метода возьмите GaussianNB ().
# В качестве метамодели выберите логистическую регрессию.
# Обучите модели и сделайте предсказание целевой метки для тестового набора данных.
# Рассчитайте F1-меру для тестового набора данных и введите её в качестве ответа, предварительно округлив до двух знаков после точки-
# разделителя.

estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=31, n_jobs=-1)), 
    ('knn', KNeighborsClassifier(n_neighbors=11, n_jobs=-1)), 
    ('nb', GaussianNB())
]
final_estimator = LogisticRegression()

stacking = StackingClassifier(
    estimators=estimators,
    final_estimator=final_estimator,
    n_jobs=-1
)

stacking.fit(X_train, y_train)
y_stack = stacking.predict(X_test)

print(f1_score(y_test, y_stack).round(2))

0.99


___

### Практика кластеризации

In [2]:
df = pd.read_csv('./data/snsdata.csv')
df.head()

,gradyear,gender,age,friends,basketball,football,soccer,softball,volleyball,swimming,...,blonde,mall,shopping,clothes,hollister,abercrombie,die,death,drunk,drugs
0,2006,M,18.982,7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2006,F,18.801,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,2006,M,18.335,69,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,2006,F,18.875,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2006,NaN,18.995,10,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,1,1


In [6]:
# Задание 4.1

# Мы будем реализовывать алгоритм только на признаках, которые заключают в себе ключевые слова, встречающиеся в сообщениях подростка.
# Удалите все остальные признаки. Какова размерность вашего текущего набора данных?

data = df.iloc[:, 4:]
data.shape

(30000, 36)

In [16]:
# Задание 4.2

# Теперь нормализуйте данные с помощью StandardScaler ( ) . Для получившейся матрицы нормализованных признаков выведите самое первое
# значение в самой первой строке (X norm 01 Г 01). Ответ округлите до двух знаков после точки-разделителя.

scaller = StandardScaler()
X_norm = scaller.fit_transform(data)
print(round(X_norm[0][0],2))


-0.33


In [30]:
# Задание 4.3

# Реализуйте алгоритм k-means с девятью кластерами. В качестве значения параметра random state возьмите число 42. Выведите
# получившееся значение инерции (model.inertia ). Ответ округлите до двух знаков после точки-разделителя.

km_segm = KMeans(n_clusters=9, random_state=42)
prediction = km_segm.fit_predict(X_norm)

print(km_segm.inertia_)


857169.1033997748


In [32]:
# Задание 4.4

# Сколько подростков относится к наиболее многочисленному кластеру?
# А сколько - к самому малочисленному?
data_km = data.copy()
data_km['cluster'] = prediction
data_km['cluster'].value_counts(ascending=True)

2        1
5      493
8      716
6      762
4      851
1      930
7     2098
3     4490
0    19659
Name: cluster, dtype: int64

In [ ]:
# Задание 4.5

# 1. Мы выделили девять кластеров, но, возможно, это далеко не оптимальное количество. Давайте попробуем применить разные методы для
# нахождения числа кластеров. Будем выбирать количество кластеров из диапазона от 3 до 9.
# Для начала примените метод локтя . Каким в этом случае будет оптимальное число кластеров? Если оно не идентифицируется однозначно,
# запишите количество кластеров, при котором достигается наименьшая инерция.


In [ ]:
# 2. Теперь попробуйте подобрать оптимальное количество кластеров с помощью коэффициента силуэта. Каким оно будет?
# Если невозможно однозначно определить количество кластеров, при котором достигается максимальное значение коэффициента силуэта,
# выберите наименьшее количество кластеров из подходящих.
